In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
# raw_data = pd.read_excel('Soal Decision Tree.xlsx')
raw_data = pd.read_csv('dataset.csv')
raw_values = raw_data.values
raw_data

,penghasilan bulanan (jt),status pernikahan,jumlah tanggungan (orang),DP,resiko
0,3.5,belum,0,5.0,rendah
1,2.2,belum,0,10.0,rendah
2,3.7,belum,0,3.0,rendah
3,5.0,belum,0,2.0,rendah
4,3.8,sudah,1,2.0,rendah
5,4.2,sudah,3,0.5,rendah
6,2.7,sudah,2,7.5,rendah
7,2.9,belum,0,5.0,rendah
8,2.3,sudah,0,9.0,rendah
9,2.0,belum,0,8.0,rendah


In [ ]:
header = ["penghasilan bulanan", "status pernikahan", "jumlah tanggungan", "DP", "resiko"]

In [3]:
def dataCounter(arrData):
    key, counts = np.unique(arrData, return_counts=True)
    result = dict(zip(key, counts))
    return result

In [4]:
def calculateTotalEntrophy(rows):
    calculate_class = dataCounter(rows)
    total_class = sum(calculate_class.values())
    entrophy_total = 0
    for key in calculate_class:
        entrophy_total += (( -1*calculate_class[key]/total_class) * math.log2((calculate_class[key]/total_class)) )
    
    return entrophy_total

In [44]:
def calculateGain(data, column_name, class_column):
    attribut = dataCounter(data[column_name])
    gain = 0
    for key in attribut:
        entrophy = 0
        attribut_class = dataCounter(data.loc[data[column_name] == key, class_column])
#         print(key, attribut[key], attribut_class)
        for k in attribut_class:
            if (attribut_class[k] != 0):
                entrophy += (( -1*attribut_class[k]/attribut[key]) * math.log2((attribut_class[k]/attribut[key])) )
        gain += ( (attribut[key] / sum(attribut.values())) * entrophy )
    return gain

def calculateSplitInfo(data, column_name):
    splitInfo = 0
    attribut = dataCounter(data[column_name])
    total_class = sum(attribut.values())
    for key in attribut:
        splitInfo += (( -1*attribut[key]/total_class) * math.log2((attribut[key]/total_class)) )
    return splitInfo

def calculateGainRatio(data, column_name, class_column):
    gain = calculateGain(data, column_name, class_column)
#     print('gain', gain)
    totalGain = entrophy_total - gain
#     print('totalgain', totalGain)
#     splitInfo = calculateSplitInfo(raw_data, column_name)
#     print('split', splitInfo)
#     resultRatio = totalGain / splitInfo
    return [totalGain, 0, 0]

In [45]:
def calculateGainNumeric(data, column_name, class_column, treshold):
    gain = 0
    total_data = len(data[class_column])
    
    attribut = dataCounter(data.loc[data[column_name] <= treshold, class_column])
    entrophy = 0
    for k in attribut:
        if (attribut[k] != 0):
            entrophy += (( -1*attribut[k]/sum(attribut.values())) * math.log2((attribut[k]/sum(attribut.values()))) )
    gain += ( (sum(attribut.values()) / total_data) * entrophy )
    
    attribut = dataCounter(data.loc[data[column_name] > treshold, class_column])
    entrophy = 0
    for k in attribut:
        if (attribut[k] != 0):
            entrophy += (( -1*attribut[k]/sum(attribut.values())) * math.log2((attribut[k]/sum(attribut.values()))) )
    gain += ( (sum(attribut.values()) / total_data) * entrophy )
    
    return gain

def calculateSplitInfoNumeric(data, column_name, treshold):
    splitInfo = 0
    total_class = sum(dataCounter(data[column_name]).values())
    
    attribut = dataCounter(data.loc[data[column_name] <= treshold, column_name])
    splitInfo += (( -1*sum(attribut.values())/total_class) * math.log2((sum(attribut.values())/total_class)) )
    
    attribut = dataCounter(data.loc[data[column_name] > treshold, column_name])
    splitInfo += (( -1*sum(attribut.values())/total_class) * math.log2((sum(attribut.values())/total_class)) )
    
    return splitInfo

def calculateGainRatioNumeric(data, column_name, class_column):
    attribut = list(dataCounter(data[column_name]).keys())
    maxTotalGain = 0
    bestTreshold = 0
    resultRatio = 0
#     print(attribut)
    for treshold in attribut[:-1]:
        gain = calculateGainNumeric(data, column_name, class_column, treshold)
        totalGain = entrophy_total - gain
        
        if totalGain > maxTotalGain :
            maxTotalGain = totalGain
            bestTreshold = treshold
            splitInfo = calculateSplitInfoNumeric(data, column_name, treshold)
            resultRatio = totalGain / splitInfo
            
    return [maxTotalGain, resultRatio, bestTreshold]

In [46]:
def is_numeric(value):
    return (value.dtype == np.int64) or (value.dtype == np.float64)

In [47]:
entrophy_total = calculateTotalEntrophy(raw_values[:, -1])
entrophy_total

0.934068055375491

In [48]:
RULE = dict()
header = list(raw_data.keys())
dataset = raw_data.copy()

while len(header) > 1:
    print(len(dataset))
    dataResult = []
    for column in header[:-1]:
        if is_numeric(dataset[column]):
            dataResult.append(calculateGainRatioNumeric(dataset, column, header[-1]))
        else:
            dataResult.append(calculateGainRatio(dataset, column, header[-1]))

    print(header[:-1])
#     print(dataResult)
    max_value = max([row[0] for row in dataResult])
#     print(max_value)
    index = [row[0] for row in dataResult].index(max_value)
    print('best gain:', header[index])
    dataResult = np.array(dataResult)
    if (is_numeric(dataset[header[index]])):
        print('split:', dataResult[index,-1])

        options = dataCounter(dataset.loc[dataset[header[index]] <= dataResult[index,-1], header[-1]])
        if (len(options) == 1):
            RULE[header[index]] = {'<= ' + str(dataResult[index,-1]): list(options.keys())[0]}
            temp = dataset.index[dataset[header[index]] <= dataResult[index,-1]].tolist()
            dataset = dataset.drop(temp)

        options = dataCounter(dataset.loc[dataset[header[index]] > dataResult[index,-1], header[-1]])
        if (len(options) == 1):
            RULE[header[index]] = {'> ' + str(dataResult[index,-1]): list(options.keys())[0]}
            temp = dataset.index[dataset[header[index]] > dataResult[index,-1]].tolist()
            dataset = dataset.drop(temp)
            
    else:
        options = dataCounter(dataset[header[index]])
#         for opt in options:
#             opt_class = dataCounter(dataset.loc[dataset[header[index]] == opt, header[-1]])
#             print(opt_class)

    header.remove(header[index])
    print()
    
print(RULE)

20
['penghasilan bulanan (jt)', 'status pernikahan', 'jumlah tanggungan (orang)', 'DP']
best gain: DP
split: 2.5

13
['penghasilan bulanan (jt)', 'status pernikahan', 'jumlah tanggungan (orang)']
best gain: penghasilan bulanan (jt)
split: 2.9

10
['status pernikahan', 'jumlah tanggungan (orang)']
best gain: status pernikahan

10
['jumlah tanggungan (orang)']
best gain: jumlah tanggungan (orang)
split: 0.0

{'DP': {'> 2.5': 'rendah'}, 'penghasilan bulanan (jt)': {'> 2.9': 'rendah'}, 'jumlah tanggungan (orang)': {'> 0.0': 'tinggi'}}


In [ ]:
def calculateGini(row):
    counts = classCounter(row)
    impurity = 1
    for label in counts:
        prob_labels = counts[label]/float(len(row))
        impurity -= prob_labels**2
    return impurity

In [ ]:
calculateGini(raw_values)

In [ ]:
classCounter(raw_values)

In [ ]:
class Question:
    
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [ ]:
Question(0,4)

In [ ]:
def splitPartition(rows,question):
    true_seeds,false_seed = [],[]
    for row in rows:
        if question.match(row):
            true_seeds.append(row)
        else:
            false_seed.append(row)
    return true_seeds,false_seed

In [ ]:
true_row,false_row=splitPartition(raw_values,Question(1,"sudah"))

In [ ]:
true_row

In [ ]:
false_row

In [ ]:
def informationGain(left_node,right_node,uncertainy):
    gain=float(len(left_node))/len(left_node)+len(right_node)
    return uncertainy-gain*calculateGini(left_node)-1-gain*calculateGini(right_node)

In [ ]:
uncertainty=calculateGini(raw_values)
uncertainty

In [ ]:
true_row,false_row=splitPartition(raw_values,Question(1,"sudah"))

In [ ]:
true_row

In [ ]:
false_row

In [ ]:
informationGain(true_row,false_row,uncertainty)